In [1]:

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# --- Step 1: Load Your PROCESSED Data ---
try:
    train_df = pd.read_csv('../data/salary.train.processed.csv', index_col='id')
    test_df = pd.read_csv('../data/salary.test.processed.csv', index_col='id')
except FileNotFoundError:
    print("Error: Could not find the processed CSV files.")
    print("Please make sure 'salary.train.processed.csv' and 'salary.test.processed.csv' are in the './data/' folder.")
    raise

# --- Step 2: Separate Features (X) and Target (y) ---
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")

# --- Step 3: Check Class Distribution ---
print("\nClass distribution in training set:")
print(y_train.value_counts())

# --- Step 4: Create PCA Pipeline and Train Logistic Regression ---
# Pipeline with StandardScaler (to ensure consistency) and PCA
pca_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Re-apply standardization
    ('pca', PCA(n_components=0.95)),  # Keep components explaining 95% of variance
    ('log_reg', LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'))
])

print("\nTraining Logistic Regression with PCA...")
pca_pipeline.fit(X_train, y_train)
print("Model training complete!")

# Number of components selected by PCA
n_components = pca_pipeline.named_steps['pca'].n_components_
print(f"Number of PCA components selected: {n_components}")

# --- Step 5: Evaluate the Model with PCA ---
y_pred_pca = pca_pipeline.predict(X_test)
y_prob_pca = pca_pipeline.predict_proba(X_test)[:, 1]

print("\nLogistic Regression Performance (with PCA):")
report_pca = classification_report(y_test, y_pred_pca, digits=4, output_dict=True)
print("Classification Report (Dictionary):")
print(report_pca)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_pca):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_pca):.4f}")

# Convert report to DataFrame for readability
report_pca_df = pd.DataFrame(report_pca).transpose()
print("\nClassification Report (DataFrame):")
print(report_pca_df)

# --- Step 6: Compare with Logistic Regression without PCA ---
log_reg = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
print("\nTraining Logistic Regression without PCA...")
log_reg.fit(X_train, y_train)
print("Model training complete!")

y_pred = log_reg.predict(X_test)
y_prob = log_reg.predict_proba(X_test)[:, 1]

print("\nLogistic Regression Performance (without PCA):")
report = classification_report(y_test, y_pred, digits=4, output_dict=True)
print("Classification Report (Dictionary):")
print(report)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.4f}")

# Convert report to DataFrame for readability
report_df = pd.DataFrame(report).transpose()
print("\nClassification Report (DataFrame):")
print(report_df)

Training features shape: (16720, 89)
Testing features shape: (4180, 89)

Class distribution in training set:
label
0.0    9719
1.0    7001
Name: count, dtype: int64

Training Logistic Regression with PCA...
Model training complete!
Number of PCA components selected: 73

Logistic Regression Performance (with PCA):
Classification Report (Dictionary):
{'0.0': {'precision': 0.8774954627949183, 'recall': 0.8004966887417219, 'f1-score': 0.8372294372294372, 'support': 2416.0}, '1.0': {'precision': 0.7560728744939271, 'recall': 0.8469387755102041, 'f1-score': 0.7989304812834225, 'support': 1764.0}, 'accuracy': 0.8200956937799043, 'macro avg': {'precision': 0.8167841686444227, 'recall': 0.823717732125963, 'f1-score': 0.8180799592564298, 'support': 4180.0}, 'weighted avg': {'precision': 0.8262539685932561, 'recall': 0.8200956937799043, 'f1-score': 0.8210669113230329, 'support': 4180.0}}

Accuracy: 0.8201
ROC AUC: 0.9025

Classification Report (DataFrame):
              precision    recall  f1-sc

In [2]:

import pandas as pd
import optuna
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# --- 1. Load Data ---
try:
    data_train_full = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
except FileNotFoundError:
    print("Error: Could not find salary.train.processed.csv. Please check the path.")
    raise

X_full = data_train_full.drop(['label'], axis='columns')
y_full = data_train_full['label']

print(f"Using {len(y_full)} records for hyperparameter tuning (K-Fold CV)")
print("\nClass distribution in training set:")
print(y_full.value_counts())

# --- 2. Define Objective Function for Optuna ---
def objective(trial):
    """
    Objective function for Optuna to optimize hyperparameters, including PCA components.
    """
    # Suggest hyperparameters
    C = trial.suggest_float('C', 1e-4, 1e2, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])
    l1_ratio = trial.suggest_float('l1_ratio', 0, 1) if penalty == 'elasticnet' else None
    n_components = trial.suggest_float('n_components', 0.5, 0.95, step=0.05)  # PCA variance ratio

    # Create pipeline with Scaler, PCA, and Logistic Regression
    pipeline_lr = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n_components)),
        ('model', LogisticRegression(
            C=C,
            penalty=penalty,
            l1_ratio=l1_ratio,
            solver='saga',
            class_weight='balanced',
            random_state=42,
            max_iter=5000,
            n_jobs=-1
        ))
    ])

    # Evaluate pipeline with 3-fold cross-validation
    score = cross_val_score(
        pipeline_lr,
        X_full,
        y_full,
        cv=3,
        scoring='f1_weighted',
        n_jobs=-1
    )

    # Return average F1 score
    f1_avg = np.mean(score)
    return f1_avg

# --- 3. Run Optuna Study ---
print("\nStarting hyperparameter search with Optuna for Logistic Regression with PCA...")

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction='maximize', pruner=pruner)
study.optimize(objective, n_trials=50, show_progress_bar=True)

# --- 4. Display Results ---
print("\n--- Optuna Search Complete! ---")
print("Best Parameters:")
print(study.best_params)
print(f"\nBest F1-Weighted (from CV): {study.best_value:.6f}")

# --- 5. Train Final Pipeline with Best Parameters ---
print("\nTraining final pipeline with best parameters...")

best_params = study.best_params
final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params['n_components'])),
    ('model', LogisticRegression(
        C=best_params['C'],
        penalty=best_params['penalty'],
        l1_ratio=best_params.get('l1_ratio'),
        solver='saga',
        class_weight='balanced',
        random_state=42,
        max_iter=5000,
        n_jobs=-1
    ))
])

# Train final pipeline on full training data
final_pipeline.fit(X_full, y_full)
print(f"Number of PCA components selected: {final_pipeline.named_steps['pca'].n_components_}")
print("Final model training complete!")

# --- 6. Evaluate on Test Set ---
print("\nEvaluating on Test Set...")
try:
    data_test_lr = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
except FileNotFoundError:
    print("Error: Could not find salary.test.processed.csv. Please check the path.")
    raise

X_test = data_test_lr.drop(['label'], axis='columns')
y_test = data_test_lr['label']

# Predict with pipeline
y_pred = final_pipeline.predict(X_test)
y_prob = final_pipeline.predict_proba(X_test)[:, 1]

# Display results
print("\nLogistic Regression (Optuna-Tuned with PCA) Confusion Matrix:")
print(confusion_matrix(y_true=y_test, y_pred=y_pred))

report_scores_lr = classification_report(
    y_true=y_test,
    y_pred=y_pred,
    digits=6,
    output_dict=True
)
df_score_lr = pd.DataFrame(report_scores_lr).transpose()
print("\nLogistic Regression (Optuna-Tuned with PCA) Report:")
print(df_score_lr)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.6f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.6f}")

# --- 7. Compare with Original Pipeline (No PCA) ---
print("\nTraining Logistic Regression without PCA for comparison...")
no_pca_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(
        C=best_params['C'],
        penalty=best_params['penalty'],
        l1_ratio=best_params.get('l1_ratio'),
        solver='saga',
        class_weight='balanced',
        random_state=42,
        max_iter=5000,
        n_jobs=-1
    ))
])

no_pca_pipeline.fit(X_full, y_full)
y_pred_no_pca = no_pca_pipeline.predict(X_test)
y_prob_no_pca = no_pca_pipeline.predict_proba(X_test)[:, 1]

print("\nLogistic Regression (Optuna-Tuned without PCA) Confusion Matrix:")
print(confusion_matrix(y_true=y_test, y_pred=y_pred_no_pca))

report_scores_no_pca = classification_report(
    y_true=y_test,
    y_pred=y_pred_no_pca,
    digits=6,
    output_dict=True
)
df_score_no_pca = pd.DataFrame(report_scores_no_pca).transpose()
print("\nLogistic Regression (Optuna-Tuned without PCA) Report:")
print(df_score_no_pca)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_no_pca):.6f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_no_pca):.6f}")


[I 2025-10-21 23:17:04,856] A new study created in memory with name: no-name-897c9842-b007-43b5-a7fb-fe77e3d26723


Using 16720 records for hyperparameter tuning (K-Fold CV)

Class distribution in training set:
label
0.0    9719
1.0    7001
Name: count, dtype: int64

Starting hyperparameter search with Optuna for Logistic Regression with PCA...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-21 23:17:12,925] Trial 0 finished with value: 0.8075195667614167 and parameters: {'C': 0.07434655686632366, 'penalty': 'l2', 'n_components': 0.8500000000000001}. Best is trial 0 with value: 0.8075195667614167.
[I 2025-10-21 23:17:16,215] Trial 1 finished with value: 0.8067560766043008 and parameters: {'C': 43.82304751568789, 'penalty': 'l2', 'n_components': 0.75}. Best is trial 0 with value: 0.8075195667614167.
[I 2025-10-21 23:17:24,412] Trial 2 finished with value: 0.8072260915122556 and parameters: {'C': 44.61019272679545, 'penalty': 'elasticnet', 'l1_ratio': 0.5655021928913363, 'n_components': 0.8}. Best is trial 0 with value: 0.8075195667614167.
[I 2025-10-21 23:17:24,737] Trial 3 finished with value: 0.7828413507175558 and parameters: {'C': 0.00019518046202855114, 'penalty': 'l2', 'n_components': 0.55}. Best is trial 0 with value: 0.8075195667614167.
[I 2025-10-21 23:17:25,072] Trial 4 finished with value: 0.4273580268070503 and parameters: {'C': 0.0001505866431315439,

In [3]:
print(best_params)

{'C': 1.9584676089237805, 'penalty': 'elasticnet', 'l1_ratio': 0.29801867330947523, 'n_components': 0.95}


In [6]:

import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

print("--- 1. Testing LogReg with Class Weight and PCA ---")

# --- Load Data ---
try:
    data_train_full = pd.read_csv('../data/salary.train.processed.csv').set_index('id')
    data_test_lr = pd.read_csv('../data/salary.test.processed.csv').set_index('id')
except FileNotFoundError:
    print("Error: Could not find salary.train.processed.csv or salary.test.processed.csv. Please check the path.")
    raise

X_full = data_train_full.drop(['label'], axis='columns')
y_full = data_train_full['label']
X_test = data_test_lr.drop(['label'], axis='columns')
y_test = data_test_lr['label']

print(f"Training features shape: {X_full.shape}")
print(f"Testing features shape: {X_test.shape}")
print("\nClass distribution in training set:")
print(y_full.value_counts())

# --- Define Parameters ---
# Assuming best_params exists from Optuna tuning (e.g., {'C': value, 'penalty': value, 'l1_ratio': value, 'n_components': value})
try:
    best_lr_params = best_params  # From prior Optuna tuning
except NameError:
    print("Warning: best_params is not defined. Using default parameters.")
    best_lr_params = {'C': 1.0, 'penalty': 'l2', 'n_components': 0.95}  # Fallback defaults

# Filter out PCA-specific parameters for LogisticRegression
lr_params = {k: v for k, v in best_lr_params.items() if k in ['C', 'penalty', 'l1_ratio']}
n_components = best_lr_params.get('n_components', 0.95)  # Default to 95% variance if not specified

# --- Create and Train Pipeline with PCA ---
pca_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=n_components)),  # Use n_components from Optuna or default
    ('model', LogisticRegression(
        **lr_params,  # Only pass LogisticRegression-compatible parameters
        class_weight='balanced',
        solver='saga',
        max_iter=5000,
        random_state=42,
        n_jobs=-1
    ))
])

print("\nTraining Logistic Regression with PCA...")
pca_pipeline.fit(X_full, y_full)
print("Model training complete.")
print(f"Number of PCA components selected: {pca_pipeline.named_steps['pca'].n_components_}")

# --- Evaluate with PCA ---
y_pred_pca = pca_pipeline.predict(X_test)
y_prob_pca = pca_pipeline.predict_proba(X_test)[:, 1]

print("\nLogReg (Tuned + Class Weight + PCA) Report:")
report_pca = classification_report(y_test, y_pred_pca, digits=6, output_dict=True)
df_report_pca = pd.DataFrame(report_pca).transpose()
print(df_report_pca)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_pca):.6f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_pca):.6f}")
print("\nConfusion Matrix (PCA):")
print(confusion_matrix(y_true=y_test, y_pred=y_pred_pca))

# --- Compare with Pipeline without PCA ---
print("\n--- 2. Testing LogReg with Class Weight (No PCA) ---")
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(
        **lr_params,  # Same parameters for fair comparison
        class_weight='balanced',
        solver='saga',
        max_iter=5000,
        random_state=42,
        n_jobs=-1
    ))
])

print("\nTraining Logistic Regression without PCA...")
pipeline.fit(X_full, y_full)
print("Model training complete.")

# --- Evaluate without PCA ---
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)[:, 1]

print("\nLogReg (Tuned + Class Weight) Report:")
report = classification_report(y_test, y_pred, digits=6, output_dict=True)
df_report = pd.DataFrame(report).transpose()
print(df_report)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.6f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.6f}")
print("\nConfusion Matrix (No PCA):")
print(confusion_matrix(y_true=y_test, y_pred=y_pred))

--- 1. Testing LogReg with Class Weight and PCA ---
Training features shape: (16720, 89)
Testing features shape: (4180, 89)

Class distribution in training set:
label
0.0    9719
1.0    7001
Name: count, dtype: int64

Training Logistic Regression with PCA...
Model training complete.
Number of PCA components selected: 73

LogReg (Tuned + Class Weight + PCA) Report:
              precision    recall  f1-score      support
0.0            0.877328  0.799255  0.836474  2416.000000
1.0            0.754927  0.846939  0.798290  1764.000000
accuracy       0.819378  0.819378  0.819378     0.819378
macro avg      0.816128  0.823097  0.817382  4180.000000
weighted avg   0.825674  0.819378  0.820360  4180.000000

Accuracy: 0.819378
ROC AUC: 0.902507

Confusion Matrix (PCA):
[[1931  485]
 [ 270 1494]]

--- 2. Testing LogReg with Class Weight (No PCA) ---

Training Logistic Regression without PCA...
Model training complete.

LogReg (Tuned + Class Weight) Report:
              precision    recall  f1-